# EDA Basic Lesson

As usual, we import the necessary libraries.

In [ ]:
import pandas as pd
import numpy as np

## Summarizing and Computing Descriptive Statistics

> **📊 Visual Illustration Available**: See `illustrations/01_descriptive_statistics.png` for visual representations of descriptive statistics including histograms, box plots, and how statistics like mean, median, and quartiles relate to data distributions.


`Series` and `DataFrame` have a number of common mathematical and statistical methods. Most of these fall into the category of _reductions_ or _summary statistics_, methods that extract a single value (like the sum or mean) from a Series or a Series of values from the rows or columns of a DataFrame. They have built-in handling for missing data.

In [ ]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5], [np.nan, np.nan], [0.75, -1.3]],
                  index=["a", "b", "c", "d"], columns=["one", "two"])

In [ ]:
df

As always, the default `axis` is `axis=0` for `DataFrame` methods, which means that by default, the reduction takes place over rows.

The method returns a Series containing column sums.

In [ ]:
df.sum()

In [ ]:
# passing axis='columns' or axis=1 sums over the columns instead
df.sum(axis="columns")

When an entire row or column contains only missing values, the sum is 0, whereas if any value is NA, it will be skipped. This can be disabled with the `skipna` option.

In [ ]:
df.sum(skipna=False)

In [ ]:
df.sum(axis=1, skipna=False)

Some methods like `idxmin` and `idxmax`, return indirect statistics like the index value where the minimum or maximum values are attained:

In [ ]:
df.idxmax()

In [ ]:
df.idxmin()

Other methods are accumulations (as opposed to reductions):

In [ ]:
df.cumsum()

Refer to the [API reference](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html#computations-descriptive-stats) for a list of methods.

> Compute the mean across rows and columns.

`describe` produces multiple summary statistics in one shot:

In [ ]:
df.describe()

It produces different summary statistics for non-numerical data.

In [ ]:
obj = pd.Series(["c", "a", "d", "a", "b", "b", "c", "c"])

obj.describe()

`unique` returns a Numpy array of unique values in a `Series`.

In [ ]:
obj.unique()

You can also use `value_counts` to return a `Series` containing unique values as its index and frequencies as its values, ordered count in descending order.

In [ ]:
obj.value_counts()

You can also sort the result by index.

In [ ]:
obj.value_counts().sort_index()

> Sort value counts in ascending order.

## Handling Missing Data

> **📊 Visual Illustration Available**: See `illustrations/03_missing_data.png` for visualizations of missing data patterns, heatmaps, and comparison of different strategies for handling missing values.


Missing data is common in most data analysis applications. One of the goals in designing pandas was to make working with missing data as painless as possible. For example, all of the descriptive statistics on pandas objects exclude missing data by default.

The `isna` method gives us a Boolean Series with `True` where values are null:

In [ ]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])

float_data

In [ ]:
float_data.isna()

In statistics applications, NA data may either be data that does not exist or that exists but was not observed (through problems with data collection, for example). When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data collection problems or potential biases in the data caused by missing data.

The built-in Python `None` value is also treated as NA:

In [ ]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])

string_data

In [ ]:
string_data.isna()

You can use `dropna` to filter out missing data. On a Series, it returns the Series with only the non-null data and index values:

In [ ]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])

data

In [ ]:
data.dropna()

In [ ]:
# which is equivalent to this

data[data.notna()]

With DataFrame objects, things are a bit more complex. You may want to drop rows or columns which are all NA or just those containing any NAs. `dropna` by default drops any row containing a missing value:

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan], 
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])

data

In [ ]:
data.dropna()

Passing `how='all'` will only drop rows that are all NA:

In [ ]:
data.dropna(how="all")

To drop columns in the same way, pass `axis=1` or `axis='columns'`:

In [ ]:
# first let's add a column of all NaNs

data[4] = np.nan

data

In [ ]:
data.dropna(axis="columns", how="all")

Suppose you want to keep only rows containing at most a certain number of missing observations. You can indicate this with the `thresh` argument:

In [ ]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))
# set some missing values
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan

df

In [ ]:
df.dropna()

In [ ]:
df.dropna(thresh=2)

Rather than filtering out missing data, you may want to fill in the "holes" in any number of ways. Calling `fillna` with a constant replaces missing values with that value:

In [ ]:
df.fillna(0)

You can also specify different fill values for each column by passing a dict.

In [ ]:
df.fillna({1: 0.5, 2: 0})

The same interpolation methods available for reindexing can be used with `fillna`, but instead of forward filling, we will try backward filling.

In [ ]:
df.fillna(method="bfill")

In [ ]:
df.fillna(method="bfill", limit=2)

A common replacement value is the mean or median statistics:

In [ ]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])

data

In [ ]:
data.fillna(data.mean())

In [ ]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))

df.iloc[[2,4,5], 1] = np.nan
df.iloc[4:, 2] = np.nan
df.iloc[3:5, 0] = np.nan

df

> Handle the missing values in df with the following methods:
> 1. Remove rows with any missing values.
> 2. Remove rows with all missing values.
> 3. Fill missing values with forward fill.
> 4. Fill missing values with mean of the column.

## Handling Duplicates

> **📊 Visual Illustration Available**: See `illustrations/06_duplicates.png` for visualizations showing how duplicates affect datasets and the impact of different `keep` parameter options.


Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [ ]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})

data

The dataframe method `duplicated` returns a boolean Series indicating whether each row is a duplicate or not:

In [ ]:
data.duplicated()

`drop_duplicates` returns a DataFrame where the duplicated array is `False`:

In [ ]:
data.drop_duplicates()

You can specify any subset of columns to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicates only based on the 'k1' column:

In [ ]:
data["v1"] = range(7)

data

In [ ]:
data.drop_duplicates(subset=["k1"])

`duplicate` and `drop_duplicates` by default keep the first observed value combination. Passing `keep='last'` will return the last one:

In [ ]:
data.drop_duplicates(subset=["k1", "k2"], keep="last")

## Transforming Data

A common step in data cleaning is transforming data based on the values in an array, Series, or column in a DataFrame.

In [ ]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon", "pastrami", 
                                "corned beef", "bacon", "pastrami", "honey ham", 
                                "nova lox"],
                         "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

data

Suppose you want to add a column indicating the animal type that each food came from. Let's map the food type to the animal type by creating a dictionary:

In [ ]:
meat_to_animal = {
    "bacon": "pig",
    "pulled pork": "pig",
    "pastrami": "cow",
    "corned beef": "cow",
    "honey ham": "pig",
    "nova lox": "salmon"
}

The `map` method on a Series accepts a function or dict-like object containing a mapping.

In [ ]:
data["animal"] = data["food"].map(meat_to_animal)

data

Passing a function works too:

In [ ]:
def get_animal(x):
    return meat_to_animal[x]

data["food"].map(get_animal) ## a functiona that applies to each element of the Series

`replace` provides a more generic way to do element-wise replacement:

In [ ]:
data = pd.Series([1, -999, 2, -999, -1000, 3])

data

The `-999` values might be sentinel values for missing data. To replace these with NA values:

In [ ]:
data.replace(-999, np.nan)

You can also replace multiple values at once by passing a list:

In [ ]:
data.replace([-999, -1000], np.nan)

To use a different replacement for each value, pass a list of substitutes or a dict:

In [ ]:
data.replace([-999, -1000], [np.nan, 0])

In [ ]:
data.replace({-999: np.nan, -1000: 0})

In [ ]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))

df.iloc[2:, 1] = -999
df.iloc[4:, 2] = 999

df

> Replace -999 with NaN and 999 with 0.

## Renaming Axis Indexes

Axis labels (index or columns) can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects.

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [ ]:
data

You can use the `map` method like a Series.

In [ ]:
def transform(x):
    return x[:4].upper()

data.index.map(transform)

Assigning back to the index will modify the DataFrame in-place:

In [ ]:
data.index = data.index.map(transform)

In [ ]:
data

You can also use the `rename` method which accepts a function or dict-like object providing a mapping.

In [ ]:
data.rename(index=str.title, columns=str.upper)

In [ ]:
data.rename(index={"OHIO": "INDIANA"}, columns={"three": 3})

## Handling Outliers

> **📊 Visual Illustration Available**: See `illustrations/04_outliers.png` for visualizations of outlier detection methods (IQR, Z-score) and comparison of different handling strategies.


Consider a DataFrame with some normally distributed data:

In [ ]:
data = pd.DataFrame(np.random.standard_normal((1000, 4)))

data

In [ ]:
data.describe()

Let's assume that values exceeding 3 in absolute value are outliers (the threshold value can be adjusted accordingly). To find the values in one of the columns exceeding 3 or -3:

In [ ]:
col = data[2]

col[col.abs() > 3]


To select all rows having a value exceeding 3 or -3, you can use the `any` method on a Boolean DataFrame:

In [ ]:
# The parentheses around data.abs() > 3 are necessary in order to call the any method on the result of the comparison operation
data[(data.abs() > 3).any(axis="columns")]

To deal with outliers, you can cap them to a maximum or minimum value. Here `np.sign` returns an array of 1 and -1 depending on the sign of the values:

In [ ]:
data[data.abs() > 3] = np.sign(data) * 3

In [ ]:
data.describe()

Another common way to deal with outliers is to drop rows that contain them.

In [ ]:
data[(data.abs() < 3).all(axis="columns")]

> Why `all` instead of `any`?
>
> Change the cap values to 2 and -2 instead.

## Permutation and Random Sampling

Permuting (randomly reordering) a Series or the rows in a DataFrame is possible using the `numpy.random.permutation` function. Calling `permutation` with the length of the axis you want to permute produces an array of integers indicating the new ordering:

In [ ]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))

df

In [ ]:
sampler = np.random.permutation(5)

sampler

You can then use `iloc` indexing or `take` method to reorder your data based on the randomly generated integer array:

In [ ]:
df.iloc[sampler]

In [ ]:
df.take(sampler)

Permuting columns work too:

In [ ]:
column_sampler = np.random.permutation(df.shape[1])

column_sampler

In [ ]:
df.take(column_sampler, axis=1)

To select a random subset without replacement (the same row cannot appear twice), you can use the `sample` method on Series and DataFrame:

In [ ]:
df.sample(n=3)

You can also sample with replacement (allowing repeat choices), by passing `replace=True`:

In [ ]:
choices = pd.Series([5, 7, -1, 6, 4])

choices.sample(n=10, replace=True)

> Sample `df` using the parameter `frac` instead of `n`.

## String Manipulation

Cleaning up a messy dataset for analysis often requires a lot of string manipulation. To complicate matters, a column containing strings will sometimes have missing data:

In [ ]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com", "Wes": np.nan}

data = pd.Series(data)

data

String and regular expression methods can be applied (passing a lambda or other function) to each value using `map`, but it will fail on the NA values. To cope with this, Series has array-oriented methods for string operations that skip NA values. These are accessed through Series's `str` attribute.

In [ ]:
data.str.contains("gmail")

Note that the result of this operation has an `object` dtype. 

Pandas has `extension types` that provide for specialized treatment of strings, integers and boolean data. We discussed `StringDType` in the previous lesson. Refer to the [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes) for more information.

In [ ]:
data_as_string = data.astype('string')

data_as_string

In [ ]:
data_as_string.str.contains("gmail")

You can also slice strings using this syntax:

In [ ]:
data_as_string.str[:5]

Regular expressions can be used, too, along with any `re` options like `IGNORECASE`:

In [ ]:
import re

In [ ]:
# regex email pattern
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"

data.str.findall(pattern, flags=re.IGNORECASE)

There are a couple of ways to do vectorized element retrieval. Either use `str.get` or index into the `str` attribute:

In [ ]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]

matches

In [ ]:
matches.str.get(1)

The `extract` method will return the captured groups of a regular expression as a DataFrame:

In [ ]:
data.str.extract(pattern, flags=re.IGNORECASE)

Refer to the [API reference](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#string-handling) for a list of string methods.

> Get the 1st group of the regex email pattern.
>
> Convert data into titlecase.

## Categorical Data

> **📊 Visual Illustration Available**: See `illustrations/05_categorical_binning.png` for visualizations comparing `pd.cut()` vs `pd.qcut()`, categorical operations, and dummy variables.


Pandas has a `categorical` type for string values with a smaller set of distinct values that are repeated. Consider the following example:

In [ ]:
values = pd.Series(["apple", "orange", "apple", "apple"] * 2)

values

In [ ]:
values.unique()

In [ ]:
values.value_counts()

Many data systems (for data warehousing, statistical computing, or other uses) have developed specialized approaches for representing data with repeated values for more efficient storage and computation. In data warehousing, a best practice is to use so-called dimension tables containing the distinct values and storing the primary observations as `integer keys` referencing the dimension table:

In [ ]:
values = pd.Series([0, 1, 0, 0] * 2)

dim = pd.Series(["apple", "orange"])

values

In [ ]:
dim

We can use `take` to restore the original Series of strings:

In [ ]:
dim.take(values)

This representation as integers is called the _category codes_ or simply _codes_. The array of distinct values can be called the _categories_.

The categorical representation can yield significant performance improvements when you are doing analytics. You can also perform transformations on the categories while leaving the codes unmodified.

Pandas' `categorical` type holds data that uses the integer-based categorical representation or encoding.

In [ ]:
fruits = ["apple", "orange", "apple", "apple"] * 2
N = len(fruits)

# to ensure reproducibility
rng = np.random.default_rng(seed=12345)

df = pd.DataFrame({"fruit": fruits, 
                   "basket_id": np.arange(N), 
                   "count": rng.integers(3, 15, size=N), 
                   "weight": rng.uniform(0, 4, size=N)}, 
                  columns=["basket_id", "fruit", "count", "weight"])

df

In [ ]:
fruit_cat = df['fruit'].astype('category')

fruit_cat

The values for `fruit_cat` are now an instance of `pandas.Categorical`, which you can access via the `.array` attribute:

In [ ]:
c = fruit_cat.array

type(c)

In [ ]:
c.categories

In [ ]:
c.codes

A trick to get mapping between codes and categories is:

In [ ]:
dict(enumerate(c.categories))

In [ ]:
df['fruit'] = df['fruit'].astype('category')

df

If you have obtained categorical encoded data from another source, you can use the `pd.Categorical.from_codes` constructor:

In [ ]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]

my_cats = pd.Categorical.from_codes(codes, categories)

my_cats

Unless explicitly specified, categorical conversions assume no specific ordering of the categories. So the `categories` array may be in a different order depending on the ordering of the input data. When using `from_codes` or any of the other constructors, you can indicate that the categories have a meaningful ordering:

In [ ]:
ordered_cats = pd.Categorical.from_codes(codes, categories, ordered=True)

ordered_cats

The output `[foo < bar < baz]` indicates that `'foo'` precedes `'bar'` in the ordering, and so on. 

### Computations with Categoricals

Continuous data is often discretized or otherwise separated into “bins” for analysis. Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Assuming you want to divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To do so, you can use `cut`:

In [ ]:
bins = [18, 25, 35, 60, 100]

age_cat = pd.cut(ages, bins)

age_cat

The object returned is `categorical`. Each bin is identified by a special (unique to pandas) `interval` value type containing the lower and upper limit of each bin:

In [ ]:
age_cat.codes

In [ ]:
age_cat.categories

In [ ]:
age_cat.categories[0]

In [ ]:
pd.value_counts(age_cat)

In the string representation of an interval, a parenthesis means that the side is open (exclusive), while the square bracket means it is closed (inclusive). You can change which side is closed by passing `right=False`:

In [ ]:
pd.cut(ages, bins, right=False)

You can override the default interval-based bin labeling by passing a list or array to the `labels` option:

In [ ]:
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]

pd.cut(ages, bins, labels=group_names)

If you pass an integer number of bins to pandas.cut instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data. 

In [ ]:
data = np.random.uniform(size=20)

pd.cut(data, 4, precision=2)

The `precision=2` option limits the decimal precision to two digits.

> Cut `ages` into 5 bins instead of 4. Set the labels to `['Youth', 'YoungAdult', 'MiddleAged', 'Senior', 'Elderly']`.

Series containing categorical data have several special methods similar to the `Series.str` specialized string methods. This also provides convenient access to the categories and codes. 

The special accessor attribute `cat` provides access to categorical methods:

In [ ]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')

cat_s

In [ ]:
cat_s.cat.codes

In [ ]:
cat_s.cat.categories

Suppose that we know the actual set of categories for this data extends beyond the four values observed in the data. We can use the `set_categories` method to change them:

In [ ]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)

cat_s2

While it appears that the data is unchanged, the new categories will be reflected in operations that use them. For example, `value_counts` respects the categories, if present:


In [ ]:
cat_s.value_counts()

In [ ]:
cat_s2.value_counts()

Conversely, you can remove categories not in your data by calling `remove_unused_categories`:

In [ ]:
cat_s2.cat.remove_unused_categories()

### Computing Indicator / Dummy Variables

Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a _dummy_ or _indicator_ matrix. If a column in a DataFrame has `k` distinct values, you would derive a matrix or DataFrame with k columns containing all 1s and 0s.

In [ ]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"], 
                   "data1": range(6)})

df

In [ ]:
pd.get_dummies(df["key"])

You can also add a prefix to the columns:

In [ ]:
dummies = pd.get_dummies(df["key"], prefix="key")

dummies

A useful recipe for statistical applications is to combine `get_dummies` with a discretization function like `cut`:

In [ ]:
np.random.seed(12345)

values = np.random.uniform(size=10)

values

In [ ]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

pd.get_dummies(pd.cut(values, bins))

> Cut values into 4 bins and create dummy variables.

## Reading / Writing data

Reading and writing data is an essential part of data analysis. Pandas has built-in methods for reading and writing data in different formats. It is also known as data input/output (IO).

### Text files

Pandas features a number of functions for reading (writing) tabular data from (to) text files as DataFrames. You can use `read_csv` to read comma-separated values (CSV) files (which are often exported from spreadsheets or databases). 

For a full list of input/output functions, see [the pandas documentation](http://pandas.pydata.org/pandas-docs/stable/io.html). The optional arguments for these functions may fall into a few categories:

* **Indexing**: You can treat one or more columns as the index of the DataFrame, or you can let pandas create a default integer index.
* **Type inference and data conversion**: This includes the user-defined value conversions and custom list of missing value markers.
* **Datetime parsing**: Includes combining capability, including combining date and time information spread over multiple columns into a single column in the result.
* **Iterating**: Support for iterating over chunks of very large files.
* **Unclean data issues**: Skipping rows or a footer, comments, or other minor things like numeric data with thousands separated by commas.

Some of these functions perform type inference, because the column data types are not part of the data format. That means you don’t necessarily have to specify which columns are numeric, integer, Boolean, or string.

In [ ]:
!cat ../data/ex1.csv

In [ ]:
df = pd.read_csv("../data/ex1.csv")

df

A file will not always have a header row.

In [ ]:
!cat ../data/ex2.csv

To read this file, you have a couple of options. You can allow pandas to assign default column names, or you can specify names yourself:

In [ ]:
pd.read_csv("../data/ex2.csv", header=None)

In [ ]:
pd.read_csv("../data/ex2.csv", names=["a", "b", "c", "d", "message"])

Suppose you wanted the `message` column to be the index of the returned DataFrame. You can either indicate you want the column at index 4 or named `"message"` using the `index_col` argument:

In [ ]:
names = ["a", "b", "c", "d", "message"]

pd.read_csv("../data/ex2.csv", names=names, index_col="message")

In some cases, a table might not have a fixed delimiter, using whitespace or some other pattern to separate fields. Consider a text file that looks like this:

In [ ]:
!cat ../data/ex3.txt

In [ ]:
result = pd.read_csv("../data/ex3.txt", sep="\s+")

result

Because there was one fewer column name than the number of data rows, `read_csv` infers that the first column should be the DataFrame’s index in this special case.

The file parsing functions have many additional arguments to help you handle the wide variety of exception file formats that occur. For example, you can skip the first, third, and fourth rows of a file with `skiprows`:

In [ ]:
!cat ../data/ex4.csv

In [ ]:
pd.read_csv("../data/ex4.csv", skiprows=[0, 2, 3])

Handling missing values is an important and frequently nuanced part of the file reading process. Missing data is usually either not present (empty string) or marked by some sentinel (placeholder) value. By default, pandas uses a set of commonly occurring sentinels, such as `NA` and `NULL`:

In [ ]:
!cat ../data/ex5.csv

In [ ]:
result = pd.read_csv("../data/ex5.csv")

result

In [ ]:
result.isna()

`read_csv` has a list of many default NA value representations, but these defaults can be disabled with the `keep_default_na` option:

In [ ]:
result2 = pd.read_csv("../data/ex5.csv", keep_default_na=False, na_values=["NA"])

result2

In [ ]:
result2.isna()

Different NA sentinels can be specified for each column in a dictionary:

In [ ]:
sentinels = {"message": ["NULL", "NA"], "something": ["two"]}

pd.read_csv("../data/ex5.csv", na_values=sentinels, keep_default_na=False)

In [ ]:
result = pd.read_csv("../data/ex3.txt", sep="\s+")

result

Using DataFrame’s `to_csv` method, we can write the data out to a comma-separated file:

In [ ]:
result.to_csv("../data/out.csv")

In [ ]:
!cat ../data/out.csv

You can also avoid writing the index:

In [ ]:
result.to_csv("../data/out.csv", index=False)

In [ ]:
!cat ../data/out.csv

> Write only columns A and B to the same file using a parameter in `to_csv`.

### Binary files

`pickle` format is a format that is optimized for reading and writing in Python. It is a format that is specific to Python, and cannot be read by other languages. It is recommended only as a short-term storage format.

In [ ]:
result.to_pickle('../data/out.pkl')

In [ ]:
data = pd.read_pickle('../data/out.pkl')

data

You can read excel file too:

In [ ]:
xlsx = pd.ExcelFile("../data/Resaleflatpricesbasedonregistrationdate.xlsx")

xlsx.sheet_names

In [ ]:
xlsx.parse(sheet_name="2017")

If you are reading multiple sheets in a file, then it is faster to create the `.ExcelFile`, but you can also simply pass the filename to `.read_excel`:

In [ ]:
frame = pd.read_excel("../data/Resaleflatpricesbasedonregistrationdate.xlsx", sheet_name="2017")

frame

To write pandas data to Excel format, you must first create an `ExcelWriter`, then write data to it using the pandas object's `to_excel` method:

In [ ]:
writer = pd.ExcelWriter('../data/out.xlsx')

frame.to_excel(writer, 'Sheet1')

writer.close()


You can also pass a file path to `to_excel` and avoid the `ExcelWriter`:

In [ ]:
frame.to_excel('../data/out.xlsx')

> Sample 100 rows then write to file: `../data/samples.xlsx`.

### Databases

You can connect to any databases using the `sqlalchemy` library.

Here, let's connect to the `DuckDB` database we created in unit 4.

In [ ]:
import sqlalchemy as sqla

Let's get the absolute path of the parent (root) directory.

In [ ]:
import os 

parent_dir = os.path.abspath(os.path.pardir)

Then, create the `sqlalchemy` engine to connect to the database.

In [ ]:
engine = sqla.create_engine(f'duckdb:///{parent_dir}/data/unit-1-4.db')

You can read the whole table into a DataFrame:

In [ ]:
df = pd.read_sql('resale_flat_prices_2017', engine)

In [ ]:
df

Or SQL query:

In [ ]:
df = pd.read_sql("SELECT * FROM resale_flat_prices_2017", engine)

In [ ]:
df

Write a filtered DataFrame to a new table:

In [ ]:
df_yishun = df[df.town == "YISHUN"]

df_yishun.to_sql("yishun_flat_prices_2017", engine)

In [ ]:
engine.table_names()

> Read only flats from `BISHAN` to a new dataframe.
>
> Then, write the dataframe to a new database table `bishan_flat_prices_2017`.